## Import Autophagy Package

In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_autophagy as cra
importlib.reload(cra)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pyvista as pv
import multiprocessing as mp
from pathlib import Path
import os

# output_path = cra.get_last_output_path()
output_path = Path("out/autophagy/2023-12-11-T13-08-29")
print(output_path)
simulation_settings = cra.get_simulation_settings(output_path)
iter_0_particles = cra.get_particles_at_iter(output_path, 0)
#for col in iter_0_particles.columns:
#    print(col)
max_iter = max(cra.get_all_iterations(output_path))

In [ ]:
fig = cra.save_cluster_information_plots(output_path, max_iter, connection_distance=1.9, overwrite=True)